# LangGraph 进阶概念：中间件 (Middleware) & 人机交互 (Human-in-the-Loop)

欢迎来到进阶课程！本 Notebook 建立在 `langchain-langgraph_1.x_01.ipynb` 的基础之上，介绍构建生产级 Agent 的几个强大功能。

**你将学到：**
- **Human-in-the-Loop (人机交互)** - 暂停 Agent 以进行人工审查和批准
- **Middleware (中间件)** - 在执行的关键点修改 Agent 的行为
- **Tool Review (工具审查)** - 为敏感工具添加审批工作流
- **Dynamic Behavior (动态行为)** - 根据上下文调整 Agent 的响应

**先决条件：** 完成 `langchain-langgraph_1.x_01.ipynb`

---

> **注意：** 这些模式对于生产级 Agent 至关重要， **因为在生产环境中，安全性、合规性和用户控制是必不可少的** 。LangGraph 1.0 引入了 `interrupt` 和 `Command` 等新原语，让这些模式的实现变得更加直观和强大。

---

## 🔄 快速参考：1.0 版本的关键新特性

本 Notebook 重点展示 LangGraph 1.0 的以下新特性：

### 1. 动态 `interrupt()` 替代静态配置

```python
# ❌ 旧版 (0.x) - 在编译时静态指定
graph = builder.compile(interrupt_before=["node_name"])

# ✅ 新版 (1.0) - 在运行时动态中断
from langgraph.types import interrupt

@tool
def sensitive_action():
    approval = interrupt({"message": "需要批准吗?"})
    if approval["approved"]:
        return "已批准"
```

**优势**：可以基于运行时条件（如置信度）动态决定是否中断。

---

### 2. `Command` 对象统一状态更新和路由

```python
# ❌ 旧版 (0.x) - 状态和路由分离
def node(state):
    return {"key": "value"}  # 只能返回状态

# ✅ 新版 (1.0) - 用 Command 同时指定状态和路由
from langgraph.types import Command

def node(state):
    return Command(
        update={"key": "value"},
        goto="next_node"
    )
```

**用于恢复中断**：
```python
agent.invoke(Command(resume={"approved": True}), config=config)
```

---

### 3. Middleware 机制（LangChain 1.0 新增）

```python
# ✅ 新版 (1.0) - 使用中间件
from langchain.agents.middleware import PIIMiddleware

agent = create_agent(
    model,
    tools,
    middleware=[
        PIIMiddleware("email", strategy="redact")
    ]
)
```

**核心 Hooks**：
- `before_model` / `after_model` - 在 LLM 调用前后执行
- `wrap_model_call` / `wrap_tool_call` - 拦截并修改调用

---

---

## 🔷 LangGraph 1.0 主要变化

### 1. **核心 API 稳定性承诺**

**变化**：LangGraph 1.0 标志着核心 API 的稳定，承诺不会有破坏性变化。

**稳定的核心原语**：
- `StateGraph` - 状态图构建器
- `add_node()` / `add_edge()` / `add_conditional_edges()` - 图构建方法
- `compile()` - 图编译
- `invoke()` / `stream()` - 执行方法
- `Checkpointer` 接口 - 持久化

**迁移建议**：
- 如果你在 0.x 使用了实验性 API，现在应迁移到稳定 API
- 查阅官方迁移指南确认你使用的 API 是否稳定

---

### 2. **`create_react_agent` 弃用**

**变化**：`langgraph.prebuilt.create_react_agent` 现已被 LangChain 的 `create_agent` 取代。

```python
# ❌ 旧版 (LangGraph 0.x)
from langgraph.prebuilt import create_react_agent
agent = create_react_agent(model, tools, messages_modifier=system_message)

# ✅ 新版 (LangChain 1.0 / LangGraph 1.0)
from langchain.agents import create_agent
agent = create_agent(model, tools, system_prompt="You are helpful")
```

**为什么弃用？**
- **架构重组**：Agent 抽象应属于 LangChain 层，LangGraph 专注于图执行引擎
- **功能增强**：新 API 支持中间件，提供更强的可定制性
- **简化依赖**：减少 LangGraph 对高层抽象的依赖

---

### 3. **新的中断机制：`interrupt()`**

**变化**：引入动态 `interrupt()` 函数，取代静态的 `interrupt_before` 配置。

```python
# ❌ 旧版 (LangGraph 0.x) - 静态配置
from langgraph.graph import StateGraph

builder = StateGraph(State)
builder.add_node("risky_action", risky_action_node)
graph = builder.compile(
    interrupt_before=["risky_action"]  # 编译时指定
)

# ✅ 新版 (LangGraph 1.0) - 动态中断
# LangGraph0.6.x 使用 `NodeInterrupt` 异常 实现动态中断，不够友好
from langgraph.types import interrupt

@tool
def risky_action():
    # 可以基于条件动态决定是否中断
    if needs_approval():
        response = interrupt({
            "message": "需要批准",
            "action": "..."
        })
        if not response["approved"]:
            return "操作取消"
    return "操作完成"
```

**动态中断的优势**：
- **条件性**：基于运行时逻辑决定是否中断（如"置信度低时才中断"）
- **上下文传递**：可以向用户传递任意数据
- **灵活控制**：在工具、节点内部任意位置调用

---

### 4. **`Command` 对象统一状态更新和路由**

**变化**：节点现在可以返回 `Command` 对象，同时指定状态更新和路由目标。

```python
# ❌ 旧版 (LangGraph 0.x) - 分离的状态和路由
def node(state):
    # 只能返回状态更新
    return {"messages": [...]}

def route_fn(state):
    # 需要单独的函数决定下一步
    if condition:
        return "next_node"
    return "end"

# ✅ 新版 (LangGraph 1.0) - Command 统一两者
from langgraph.types import Command

def node(state):
    # 可以同时指定状态更新和路由
    return Command(
        update={"messages": [...]},
        goto="next_node"
    )
```

**`Command` 对象的威力**：
```python
# 恢复中断时也使用 Command
## LangGraph0.6.X版本使用 `None` 调用图时，将从最后一个状态检查点继续执行！
agent.invoke(
    Command(resume={"approved": True}),
    config=config
)

# 多目标路由（广播）
return Command(
    update={...},
    goto=["node1", "node2"]  # 并行执行
)
```

---

### 5. **Checkpointer 接口统一**

**变化**：统一了持久化接口，废弃了旧版混乱的配置方式。

```python
# ✅ 标准 Checkpointer 使用方式
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver

# 开发环境
checkpointer = MemorySaver()

# 生产环境
checkpointer = SqliteSaver.from_conn_string("sqlite:///checkpoints.db")

agent = create_agent(
    model,
    tools,
    checkpointer=checkpointer
)
```

**支持的 Checkpointer**：
- `MemorySaver` - 内存（开发/测试）
- `SqliteSaver` - SQLite 数据库
- `PostgresSaver` - PostgreSQL（生产推荐）
- 自定义实现 `BaseCheckpointSaver` 接口
---


### LangGraph 1.0 升级指南 - 人机交互与控制流

在 LangGraph 1.0 中，**Human-in-the-Loop** 和 **控制流** 发生了重大变化，变得更加动态和灵活。

**主要区别：**

#### 1. 中断机制：`interrupt()` vs 静态配置
- **旧版 (0.x)**: 需要在编译 Graph 时通过 `interrupt_before=["node_name"]` 静态指定在哪停。
- **新版 (1.0)**: 引入了动态 **`interrupt()`** 函数。你可以在任何节点或工具内部调用它。这允许基于逻辑（如“置信度低时才中断”）动态决定是否暂停。

#### 2. 状态更新与导航：`Command` 对象
- **旧版 (0.x)**: 节点返回状态更新，路由由单独的条件边函数处理。
- **新版 (1.0)**: 引入 **`Command`** 对象。一个节点可以同时返回状态更新 (`update`) 和路由指令 (`goto`)，极大简化了复杂逻辑的编写。

本教程的“第 1 部分”和“第 2 部分”将详细演示这些新特性的用法。

## 前置部分：环境配置和模型配置

### 创建langchain1.x 环境（在shell中执行）
```
# 创建langchain1.x 环境
conda create -n langchain1.x python=3.10.18 -y

# 确保在 langchain1.x 环境中
conda activate langchain1.x

# 安装 ipykernel
pip install ipykernel

# 将环境注册为 Jupyter 内核
python -m ipykernel install --user --name=langchain1.x --display-name="Python (langchain1.x)"

# 验证内核安装
jupyter kernelspec list
```

### 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助你：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(langchain1.x)`，即可执行下方代码

In [1]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate langchain1.x

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "langchain1.x" ]; then
    echo "✅ 当前单元格已成功激活到 langchain1.x 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(langchain1.x)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(langchain1.x)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate langchain1.x"
fi

echo "=========================================" 

== Conda 环境检查报告 (仅针对当前 Bash 子进��) ==
✅ 当前单元格已成功激活到 langchain1.x 环境。
✅ 正在使用的环境路径: /root/miniconda3/envs/langchain1.x

💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。
   如果需要后续单元格也使用此环境，请执行以下操作:
   1. 检��� Notebook 右上角是否已选择 'python(langchain1.x)'。


### 🔍 环境信息打印

In [2]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助你：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                               |
|:-------------|:-----------------------------------------------------------------------------------|
| 操作系统     | Linux Ubuntu 22.04.4 LTS                                                           |
| CPU 信息     | 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz (1 physical cores, 4 logical cores) |
| 内存信息     | 5.75 GB (Available: 4.91 GB)                                                       |
| GPU 信息     | No GPU found (nvidia-smi not found)                                                |
| CUDA 信息    | CUDA not found                                                                     |
| Python 版本  | 3.10.18                                                                            |
| Conda 版本   | conda 24.4.0                                         

## 第 0 部分: 设置与安装

首先，我们需要安装必要的 Python 包并设置环境。

In [3]:
# 安装所需的包
!pip install langchain==1.1.3 \
langchain-openai==1.1.1 \
python-dotenv==1.2.1 \
langgraph==1.0.4 \
sqlalchemy==2.0.44 \
requests==2.32.5

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


### 配置模型

In [4]:
# 导入必要的模块
import os, getpass

def _set_env(var: str):
    """
    设置环境变量的辅助函数
    如果环境变量不存在，则提示用户输入
    """
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


# sk-AGnnT4BAKJBA22uYA32fF98e9d1645739916347057D2A14f

# 设置 OpenAI API 密钥 
# 这是使用 OpenAI 模型所必需的
_set_env("OPENAI_API_KEY")
# 设置 OpenAI API代理地址 (例如：https://api.apiyi.com/v1）
_set_env("OPENAI_BASE_URL")

In [5]:
"""
模型初始化文件

此文件配置应用程序中使用的 LLM (大语言模型)。

默认配置:
- 默认提供商是 OpenAI (使用 o3-mini 模型)
- 你也可以通过取消注释相应的行来切换到 Anthropic

备选提供商:
要使用不同的 LLM 提供商，请按照以下步骤操作:
1. 注释掉下方的 "Default Models" (默认模型) 部分
2. 取消注释你想要的提供商部分:
   - Azure OpenAI: 需要设置 AZURE_OPENAI_API_KEY 和 AZURE_OPENAI_ENDPOINT
   - AWS Bedrock: 需要设置 AWS 凭证和配置
   - Google Vertex AI: 需要设置 GOOGLE_APPLICATION_CREDENTIALS
3. 按照每个部分内的设置说明进行操作
"""

# """默认模型 (Default Models)"""
from langchain.chat_models import init_chat_model
# 初始化聊天模型，这里使用 OpenAI 的 o3-mini
model = init_chat_model("openai:o3-mini")

# 使用 Anthropic 替代 OpenAI
# model = init_chat_model("anthropic:claude-haiku-4-5")


# """AZURE OpenAI Version"""
# from langchain_openai import AzureChatOpenAI
# # from langchain_anthropic import ChatAnthropic
# # from langchain_google_vertexai import ChatVertexAI
# from azure.identity import InteractiveBrowserCredential

# credential = InteractiveBrowserCredential()

# def get_token():
#     token = credential.get_token("https://cognitiveservices.azure.com/.default")
#     return token.token

# For AzureOpenAI, make sure you set AZURE_OPENAI_API_KEY and AZURE_OPENAI_ENDPOINT

# Azure OpenAI: Using Environment Variables
# AZURE_OPENAI_GPT_4O = AzureChatOpenAI(
#     azure_deployment="gpt-4o",
#     streaming=True
# )

# Azure OpenAI: Using Azure AD
# AZURE_OPENAI_GPT_4O = AzureChatOpenAI(
#     api_version="2024-03-01-preview",
#     azure_endpoint="https://deployment.openai.azure.com/",
#     azure_deployment="gpt-4o",
#     azure_ad_token_provider=get_token
# )


# """Bedrock Version"""
# from dotenv import load_dotenv
# from langchain_aws import ChatBedrockConverse
# import os

# load_dotenv(dotenv_path="../../.env", override=True)

# AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
# AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
# AWS_REGION_NAME=os.getenv("AWS_REGION_NAME")
# AWS_MODEL_ARN=os.getenv("AWS_MODEL_ARN")

# model = ChatBedrockConverse(
#     aws_access_key_id=AWS_ACCESS_KEY_ID,
#     aws_secret_access_key=AWS_SECRET_ACCESS_KEY, 
#     region_name=AWS_REGION_NAME,
#     provider="anthropic",
#     model_id=AWS_MODEL_ARN
# )


# """Google Vertex AI version"""
# Make sure you have your vertex ai credentials setup and your GOOGLE_APPLICATION_CREDENTIALS are pointing to the JSON file. 

# import os
# from pathlib import Path
# from dotenv import load_dotenv
# from langchain.chat_models import init_chat_model

# # Find project root and load .env
# # Use __file__ to get the location of this file, then go up two directories to project root
# project_root = Path.cwd().parent.parent
# load_dotenv(dotenv_path=project_root / ".env", override=True)

# # Fix credentials path to absolute
# if "GOOGLE_APPLICATION_CREDENTIALS" in os.environ:
#     cred_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
#     if not os.path.isabs(cred_path):
#         os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(project_root / cred_path.lstrip("./"))

# # Create model
# model = init_chat_model("google_vertexai:gemini-2.5-flash")


### 常用方法配置

In [6]:
# 忽略 LangSmith 使用 UUID v7 的警告（在简单的无线程 ID 示例中）
import warnings
warnings.filterwarnings('ignore', message='LangSmith now uses UUID v7')

## 第 1 部分: 使用 Interrupts 实现人机交互 (Human-in-the-Loop)

### 问题背景

想象一下，你正在构建一个可以发送电子邮件或进行电子商城下单购买的 Agent。你肯定不希望它自动执行这些操作 —— 你希望先经过人工批准！

**Human-in-the-Loop** 允许你：
- 暂停执行以进行审查
- 批准、拒绝或编辑动作
- 为敏感操作添加安全控制

### 工作原理 (LangGraph 1.0 新特性)

LangGraph 1.0 引入了动态 `interrupt` 函数，替代了旧版较为静态的配置方式。

1. Agent 遇到 `interrupt()` 函数 - 执行暂停
2. 系统将信息展示给人类
3. 人类提供输入（批准/拒绝/编辑）
4. Agent 使用 `Command(resume=...)` 恢复执行，并将人类输入作为 `interrupt()` 的返回值传回

### 示例 1: 简单的审批工作流

让我们从一个简单的例子开始——在发送电子邮件之前请求批准。

---

#### 🔄 版本对比：`interrupt()` 的使用

```python
# ❌ 旧版 (LangGraph 0.x) - 需要在编译时配置
# 1. 定义节点
def send_email_node(state):
    # ... 发送邮件逻辑
    return state

# 2. 在编译时指定中断点
builder.add_node("send_email", send_email_node)
graph = builder.compile(
    interrupt_before=["send_email"]  # 静态配置
)

# 3. 运行时无法动态决定是否中断
result = graph.invoke(input, config=config)

# ✅ 新版 (LangGraph 1.0) - 在工具内部动态中断
@tool
def send_email(to: str, subject: str, body: str) -> str:
    # 可以基于条件动态决定是否需要批准
    if is_sensitive_content(body):  # 条件判断
        approval = interrupt({
            "to": to,
            "subject": subject,
            "message": "检测到敏感内容，需要批准"
        })
        if not approval["approved"]:
            return "邮件已取消"
    return f"邮件已发送至 {to}"
```

**新版优势**：
- ✅ **条件性中断**：只在需要时才中断（如"敏感内容"、"高金额"等）
- ✅ **上下文传递**：可以向审批者传递详细信息
- ✅ **无需修改图结构**：逻辑封装在工具内部

---

In [7]:
from langgraph.types import interrupt
from langchain_core.tools import tool

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """发送邮件给接收者。"""
    
    # 暂停以等待人工批准
    # interrupt 接收的参数会作为暂停时的信息返回给前端/用户
    # 程序会在这里挂起，直到收到 resume 指令
    approval = interrupt({
        "action": "send_email",
        "to": to,
        "subject": subject,
        "body": body,
        "message": "您确定要发送这封邮件吗？"
    })
    
    # 恢复执行后，approval 变量将包含 resume 时传入的数据
    if approval.get("approved"): # 如果通过批准（根据我们定义的协议）
        # 在生产环境中，这里会实际发送邮件
        return f"邮件已发送至 {to}，主题为 '{subject}'"
    else:
        return "用户取消了邮件发送"

# 测试工具定义
print("工具创建成功！")
print(f"工具名称: {send_email.name}")
print(f"工具描述: {send_email.description}")

工具创建成功！
工具名称: send_email
工具描述: 发送邮件给接收者。


### 创建带有 Human-in-the-Loop 的 Agent

现在让我们创建一个使用此工具的 Agent。**切记：** Interrupts 必须配合 Checkpointer 使用，否则无法保存暂停时的状态！

---

### 🔄 Checkpointer 的变化 (0.x vs 1.0)

#### LangGraph 1.0 统一了 Checkpointer 接口

**旧版 (0.x) 的问题**：
- 配置方式不统一
- 不同的 checkpointer 有不同的初始化参数
- 文档分散，难以选择合适的实现

**新版 (1.0) 的改进**：

```python
# ✅ 统一的接口，清晰的选择

# 1. 开发/测试环境 - 内存存储
from langgraph.checkpoint.memory import MemorySaver
checkpointer = MemorySaver()

# 2. SQLite - 本地持久化
from langgraph.checkpoint.sqlite import SqliteSaver
checkpointer = SqliteSaver.from_conn_string("sqlite:///checkpoints.db")

# 3. PostgreSQL - 生产环境推荐
from langgraph.checkpoint.postgres import PostgresSaver
checkpointer = PostgresSaver.from_conn_string("postgresql://...")

# 4. 自定义 - 实现 BaseCheckpointSaver 接口
from langgraph.checkpoint import BaseCheckpointSaver
class MyCustomSaver(BaseCheckpointSaver):
    # 实现 get(), put(), list() 方法
    pass
```

**选择指南**：
- **`MemorySaver`**：快速原型、测试、demo
- **`SqliteSaver`**：单机应用、小规模生产
- **`PostgresSaver`**：多用户、高并发、生产环境

**为什么 Checkpointer 是必须的？**
- **Interrupt 恢复**：保存暂停时的完整状态
- **多轮对话**：保持会话历史
- **容错能力**：故障后可以从中断点继续
- **时间旅行**：回退到历史状态进行调试

---

In [8]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver

# 创建 Checkpointer 用于持久化
checkpointer = MemorySaver()

# 创建带有邮件工具的 Agent
agent = create_agent(
    model=model,
    tools=[send_email],
    system_prompt="你是一个乐于助人的邮件助手。当被要求发送邮件时，请使用 send_email 工具。",
    checkpointer=checkpointer  # 必须项：用于 interrupts
)

### 运行直到中断 (Running Until Interrupt)

让我们运行 Agent，看看它如何暂停以等待批准：

In [9]:
from langchain.messages import HumanMessage
from langsmith import uuid7

# 为此对话创建唯一线程
config = {"configurable": {"thread_id": uuid7()}}

# 运行 Agent
result = agent.invoke(
    {
        "messages": [HumanMessage(content="给 alice@example.com 发送一封邮件，主题是 '明天开会'，内容是 '我们下午3点见面。'")]
    },
    config=config
)

# 检查是否触发了 interrupt
# 在 LangGraph 1.0 中，如果因 interrupt 暂停，结果中会包含 __interrupt__ 键
if "__interrupt__" in result:
    print("Agent 已暂停等待批准\n")

    # 获取 interrupt 的详细信息
    interrupt_info = result["__interrupt__"][0]

    print("中断详情：")
    print(f"  收件人: {interrupt_info.value['to']}")
    print(f"  主题: {interrupt_info.value['subject']}")
    print(f"  内容: {interrupt_info.value['body']}")
    print(f"  消息: {interrupt_info.value['message']}")
else:
    print("Agent 执行完成，未触发中断")

Agent 已暂停等待批准

中断详情：
  收件人: alice@example.com
  主题: 明天开会
  内容: 我们下午3点见面。
  消息: 您确定要发送这封邮件吗？


### 批准并恢复执行 (Resuming with Approval)

现在让我们批准这封邮件，并让 Agent 继续运行。
我们使用 `Command` 对象并传入 `resume` 参数。这个值会直接成为 `interrupt()` 函数在工具内部的返回值。

---

#### 🔄 版本对比：恢复执行的方式

```python
# ❌ 旧版 (LangGraph 0.x) - 重新 invoke 完整输入
result = graph.invoke(input, config=config)
# 中断后，需要重新传入完整的 input
result = graph.invoke(input, config=config)  # 从中断处继续

# ✅ 新版 (LangGraph 1.0) - 使用 Command 对象
from langgraph.types import Command

# 运行到中断
result = agent.invoke(input, config=config)

# 恢复 - 只需传入 Command 对象
result = agent.invoke(
    Command(resume={"approved": True}),  # 传递审批结果
    config=config  # 相同的 config（thread_id）
)
```

**`Command` 对象的强大之处**：
- ✅ **语义清晰**：`Command(resume=...)` 明确表达"恢复中断"的意图
- ✅ **无需重复输入**：不需要再次传入完整的原始 input
- ✅ **灵活的数据传递**：可以传递任意结构的审批数据

**`Command` 的其他用法**：
```python
# 1. 状态更新 + 路由
Command(update={"messages": [...]}, goto="next_node")

# 2. 恢复中断（本例）
Command(resume={"approved": True})

# 3. 多目标路由（并行）
Command(update={...}, goto=["node1", "node2"])
```

---

In [10]:
from langgraph.types import Command

# 恢复执行并给予批准
# 这里的 {"approved": True} 将会是工具中 approval 变量的值
result = agent.invoke(
    Command(resume={"approved": True}),
    config=config
)

# 打印最终响应
print("最终响应：")
print(result["messages"][-1].content)

最终响应：
邮件已成功发送！


### 练习：尝试拒绝邮件

再次运行上面的步骤，但这次通过传入 `{"approved": False}` 来拒绝邮件：

In [11]:
# 为拒绝示例创建一个新线程
config_2 = {"configurable": {"thread_id": uuid7()}}

# 运行直到中断
result = agent.invoke(
    {
        "messages": [HumanMessage(content="给 bob@example.com 发送一封邮件，内容是 '你好！'")]
    },
    config=config_2
)

# 恢复并拒绝
result = agent.invoke(
    Command(resume={"approved": False}),  # 拒绝邮件
    config=config_2
)

print("最终响应：")
print(result["messages"][-1].content)

最终响应：
邮件发送已取消。如需重新发送，请告诉我。


In [12]:
print(result["messages"])

[HumanMessage(content="给 bob@example.com 发送一封邮件，内容是 '你好！'", additional_kwargs={}, response_metadata={}, id='744da160-072c-4938-b8e2-0081ecabec53'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 94, 'total_tokens': 196, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_52e4403cb8', 'id': 'chatcmpl-Cw2DjJNp7VEx501j5qczL1Sf0UcBg', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019ba1e3-6e5a-7563-8139-dbdb3882f066-0', tool_calls=[{'name': 'send_email', 'args': {'body': '你好！', 'subject': '邮件', 'to': 'bob@example.com'}, 'id': 'call_IXrReFPzuW0s04IapK3FSOuo', 'type': 'tool_call'}], usage_metadata={'input_tokens': 94, 'output_tokens': 102, 'total_

## 第 2 部分: 进阶模式 - 执行前先编辑 (Edit Before Execution)

有时你不仅仅想批准或拒绝，还想**编辑**工具调用的参数。让我们升级我们的工具：

In [13]:
@tool
def send_email_v2(to: str, subject: str, body: str) -> str:
    """发送邮件给接收者。"""
    
    # 暂停等待人工审查
    response = interrupt({
        "action": "send_email",
        "to": to,
        "subject": subject,
        "body": body,
        "message": "请审查这封邮件。您可以批准、拒绝或编辑它。"
    })
    
    # 处理不同的响应类型
    if response["type"] == "approve":
        return f"邮件已发送至 {to}，主题为 '{subject}'"

    elif response["type"] == "reject":
        return "邮件已取消"

    elif response["type"] == "edit":
        # 更新为编辑后的值
        to = response.get("to", to)
        subject = response.get("subject", subject)
        body = response.get("body", body)
        return f"""Email sent with edits:
                To: {to}
                Subject: {subject}
                Body: {body}"""
    
    return "未知响应"

# 创建使用增强工具的新 Agent
agent_v2 = create_agent(
    model=model,
    tools=[send_email_v2],
    system_prompt="你是一个乐于助人的邮件助手。",
    checkpointer=MemorySaver()
)

In [14]:
# 运行并编辑邮件
config_3 = {"configurable": {"thread_id": uuid7()}}

# 运行直到中断
result = agent_v2.invoke(
    {
        "messages": [HumanMessage(content="给 team@example.com 发送一封关于会议的邮件")]
    },
    config=config_3
)

print("暂停等待审查...\n")

暂停等待审查...



现在，让我们修改邮件主题，将其标记为 URGENT (紧急) 会议！

In [15]:
# 恢复并带入编辑后的内容
result = agent_v2.invoke(
    Command(resume={
        "type": "edit",
        "subject": "紧急：今天下午2点开会",  # 我们修改了邮件主题
        "body": "这是编辑后的邮件内容，包含更多细节。"
    }),
    config=config_3
)

print("最终响应：")
print(result["messages"][-1].content)

最终响应：
邮件已经发送。


In [16]:
print(result["messages"])

[HumanMessage(content='给 team@example.com 发送一封关于会议的邮件', additional_kwargs={}, response_metadata={}, id='a74697db-9f52-49fe-bfd3-2163453a0d26'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 361, 'prompt_tokens': 80, 'total_tokens': 441, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'o3-mini-2025-01-31', 'system_fingerprint': 'fp_52e4403cb8', 'id': 'chatcmpl-Cw2Ds69jr82kmOip2moqGZfubxcEs', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019ba1e3-8e19-73e0-a0c7-94f36459c6b2-0', tool_calls=[{'name': 'send_email_v2', 'args': {'body': '大家好，\n\n这封邮件是关于即将召开的会议的通知。请各位安排好时间，我们将讨论目前项目的进展以及未来的计划安排。\n\n如有任何问题，请随时与我联系。\n\n谢谢！\n\n此致\n敬礼', 'subject': '关于会议的通知', 'to': 'team@example.com'}, 'id': 'call_LFDcidzMkTjKvOo8Cha

## 第 3 部分: 中间件 (Middleware) 简介

**Middleware (中间件)** 提供了对 Agent 循环 (Loop) 的细粒度控制。它可以让你：
- 在模型调用前后检查状态
- 动态修改模型请求
- 在执行的关键点添加自定义逻辑

---

### 🆕 LangChain 1.0 的核心新特性：Middleware

**中间件是 LangChain 1.0 引入的全新机制，0.x 版本不存在！**

#### 旧版如何实现类似功能？

```python
# ❌ 旧版 (0.x) - 需要手动在节点中实现所有逻辑
def agent_node(state):
    # 日志记录 - 手动实现
    print(f"处理 {len(state['messages'])} 条消息")
    
    # PII 过滤 - 手动实现
    messages = []
    for msg in state["messages"]:
        filtered_msg = redact_pii(msg)  # 自己写过滤逻辑
        messages.append(filtered_msg)
    
    # 调用模型
    response = model.invoke(messages)
    
    # 验证输出 - 手动实现
    if not is_safe(response):
        response = "我无法回答这个问题"
    
    return {"messages": [response]}

# ✅ 新版 (1.0) - 使用中间件，逻辑解耦且可复用
from langchain.agents.middleware import PIIMiddleware, before_model, after_model

@before_model
def log_middleware(state, runtime):
    print(f"处理 {len(state['messages'])} 条消息")
    return None

@after_model
def safety_middleware(state, runtime):
    last_msg = state["messages"][-1]
    if not is_safe(last_msg):
        return {"messages": [AIMessage("我无法回答这个问题")]}
    return None

agent = create_agent(
    model,
    tools,
    middleware=[
        log_middleware,
        PIIMiddleware("email", strategy="redact"),
        safety_middleware
    ]
)
```

**中间件的优势**：
- 🔁 **可复用**：一次编写，多个 Agent 共享
- 🧩 **模块化**：每个中间件负责单一职责
- 🔌 **可插拔**：轻松启用/禁用特定功能
- 🧹 **代码清晰**：横切关注点从业务逻辑中分离

---

### Agent 循环 (The Agent Loop)

```
Input --> [before_model] --> [wrap_model_call] --> Model --> [after_model] --> Tools --> ...
```

Middleware 会挂载到这个循环中：
- **`before_model`** - 在模型执行前运行，可以更新状态
- **`wrap_model_call`** - 包装模型调用，控制模型何时/如何被调用（例如重试、缓存）
- **`after_model`** - 在模型执行后，执行工具前运行

### 两种 Hook 风格

**Node-style hooks (节点式钩子)** 按顺序运行：
- `before_agent`, `before_model`, `after_model`, `after_agent`
- 适合用于日志记录、验证、状态更新

**Wrap-style hooks (包装式钩子)** 拦截执行：
- `wrap_model_call`, `wrap_tool_call`
- 完全控制处理程序的调用
- 适合用于重试逻辑、缓存、数据转换

### 📋 LangChain 1.0 Middleware 完整参考指南

#### Middleware Hook 的执行顺序

```
用户输入
    ↓
[before_agent]  ← 整个 Agent 执行前（一次）
    ↓
┌─────────────────────────── Agent 循环 ───────────────────────────┐
│                                                                   │
│  [before_model]  ← 每次 LLM 调用前                               │
│      ↓                                                            │
│  [wrap_model_call]  ← 包裹 LLM 调用（可重试、缓存）              │
│      ↓                                                            │
│  LLM 执行                                                         │
│      ↓                                                            │
│  [after_model]  ← LLM 响应后                                      │
│      ↓                                                            │
│  如果有工具调用:                                                  │
│      [wrap_tool_call]  ← 包裹每个工具调用                         │
│      工具执行                                                     │
│      返回 before_model（继续循环）                                │
│                                                                   │
└───────────────────────────────────────────────────────────────────┘
    ↓
[after_agent]  ← 整个 Agent 完成后（一次）
    ↓
最终输出
```

#### Hook 类型详解

##### 1. Node-Style Hooks（节点式钩子）

按顺序执行，可以修改状态：

| Hook | 执行时机 | 返回值 | 典型用途 |
|------|----------|--------|----------|
| `before_agent` | Agent 开始前 | `dict \| None` | 加载用户上下文、初始化状态 |
| `before_model` | 每次调用 LLM 前 | `dict \| None` | 动态修改 prompt、裁剪消息历史 |
| `after_model` | LLM 响应后 | `dict \| None` | 验证输出、应用内容过滤 |
| `after_agent` | Agent 完成后 | `dict \| None` | 保存结果、记录日志 |

```python
from langchain.agents.middleware import before_model

@before_model
def token_limit_middleware(state, runtime):
    """限制消息数量，防止超过 token 限制"""
    messages = state.get("messages", [])
    if len(messages) > 10:
        # 保留最近 10 条消息
        return {"messages": messages[-10:]}
    return None
```

##### 2. Wrap-Style Hooks（包装式钩子）

拦截并控制执行流程：

| Hook | 执行时机 | 参数 | 典型用途 |
|------|----------|------|----------|
| `wrap_model_call` | 围绕 LLM 调用 | `request`, `handler` | 重试逻辑、缓存、请求修改 |
| `wrap_tool_call` | 围绕工具调用 | `request`, `handler` | 权限检查、参数验证、模拟执行 |

```python
from langchain.agents.middleware import AgentMiddleware

class RetryMiddleware(AgentMiddleware):
    """LLM 调用失败时自动重试"""
    
    def wrap_model_call(self, request, handler):
        max_retries = 3
        for attempt in range(max_retries):
            try:
                return handler(request)
            except Exception as e:
                if attempt == max_retries - 1:
                    raise
                print(f"重试 {attempt + 1}/{max_retries}...")
                time.sleep(1)
```

#### 内置 Middleware（LangChain 1.0）

LangChain 1.0 提供了开箱即用的中间件：

| 中间件 | 功能 | 配置示例 |
|--------|------|----------|
| `PIIMiddleware` | PII 数据脱敏 | `PIIMiddleware("email", strategy="redact")` |
| `SummarizationMiddleware` | 对话历史摘要 | `SummarizationMiddleware(model, trigger={"tokens": 500})` |
| `HumanInTheLoopMiddleware` | 工具调用审批 | `HumanInTheLoopMiddleware(interrupt_on={"tool_name": {...}})` |

```python
from langchain.agents.middleware import (
    PIIMiddleware,
    SummarizationMiddleware
)

agent = create_agent(
    model,
    tools,
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True),
        PIIMiddleware(
            "phone_number",
            detector=r"\\+?\\d{1,3}[\\s.-]?\\d{2,4}[\\s.-]?\\d{3,4}",
            strategy="block"
        ),
        SummarizationMiddleware(
            model="openai:gpt-4o-mini",
            trigger={"tokens": 500}
        )
    ]
)
```

#### 自定义 Middleware 最佳实践

1. **单一职责**：每个中间件只做一件事
2. **无状态**：中间件应该是无状态的，状态保存在 Agent State 中
3. **快速失败**：错误处理要明确，该抛出异常时不要静默
4. **可配置**：通过构造函数接受配置参数
5. **文档化**：清楚说明中间件的作用和副作用

```python
class MyMiddleware(AgentMiddleware):
    """清晰的 docstring 说明用途
    
    Args:
        threshold: 阈值参数
        enabled: 是否启用
    """
    
    def __init__(self, threshold: float, enabled: bool = True):
        self.threshold = threshold
        self.enabled = enabled
    
    def before_model(self, state, runtime):
        if not self.enabled:
            return None
        # 实现逻辑...
```

---


### 示例 1: 动态系统提示词 (Dynamic System Prompt)

让我们创建一个中间件，根据用户的角色动态更改系统提示词：

In [17]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from typing import TypedDict

# 定义上下文 Schema
class Context(TypedDict):
    user_role: str

# 使用装饰器创建中间件
@dynamic_prompt
def dynamic_prompt_middleware(request: ModelRequest) -> str:
    """根据用户角色调整系统提示词。"""
    
    # 从运行时上下文中获取 user_role
    user_role = request.runtime.context.get("user_role", "general")
    
    if user_role == "expert":
        return "你是一名面向专家的 AI 助手。请提供包含代码示例的详细技术解答。"
    elif user_role == "beginner":
        return "你是一名面向初学者的 AI 助手。请通俗易懂地解释概念，避免使用行话。"
    else:
        return "你是一名乐于助人的 AI 助手。"

In [18]:
from langchain_core.tools import tool

@tool
def explain_concept(concept: str) -> str:
    """解释一个编程概念。"""
    explanations = {
        "async": "异步编程允许代码在不阻塞的情况下运行。",
        "recursion": "递归是指函数调用自身的情况。"
    }
    return explanations.get(concept.lower(), "未找到该概念。")

# 创建带有中间件的 Agent
agent_with_middleware = create_agent(
    model=model,
    tools=[explain_concept],
    middleware=[dynamic_prompt_middleware],
    context_schema=Context
)

### 测试不同的用户角色

让我们看看 Agent 如何根据用户角色做出不同的反应：

In [19]:
# 专家用户
print("=" * 50)
print("专家用户")
print("=" * 50)

result = agent_with_middleware.invoke(
    {"messages": [HumanMessage(content="解释异步编程")]},
    context={"user_role": "expert"}
)
print(result["messages"][-1].content)
print()

# 初学者
print("=" * 50)
print("初学者")
print("=" * 50)

result = agent_with_middleware.invoke(
    {"messages": [HumanMessage(content="解释异步编程")]},
    context={"user_role": "beginner"}
)
print(result["messages"][-1].content)

专家用户
异步编程是一种编程模型，用于处理耗时任务而不阻塞主线程的执行。传统的同步模型中，程序必须等待一个操作（如网络请求、文件IO）完成后才能继续执行后续代码，而异步编程则允许这些操作在后台执行，让主程序可以继续响应其他事件，从而提升整体性能和用户体验。

以下通过JavaScript来详细说明异步编程的关键概念和实现方式：

1. Promise

Promise 是一种用于封装异步操作的对象，表示一个可能还未完成但将在未来某个时间点完成的操作。Promise 提供了 .then()、.catch() 等方法，方便我们在异步操作成功或失败后执行相应的处理逻辑。

示例代码：
-------------------------------------------------
function fetchData() {
    return new Promise((resolve, reject) => {
        // 模拟一个耗时操作，比如网络请求或定时任务
        setTimeout(() => {
            const data = { id: 1, name: 'Alice' };
            resolve(data);  // 操作成功，返回数据
        }, 1000);
    });
}

fetchData()
    .then(data => {
        console.log('Promise获取到的数据:', data);
    })
    .catch(error => {
        console.error('发生错误:', error);
    });
-------------------------------------------------

2. async/await

async/await 是基于 Promise 实现的一种更直观的异步编程方式。通过 async 声明一个函数为异步函数，在该函数内部可以使用 await 等待 Promise 完成，使得异步代码风格类似同步代码，提升了代码的可读性和易维护性。

示例代码：
-------------------------------------------------
async fu

### 示例 2: 自定义中间件 - 请求日志记录器 (Request Logger)

中间件允许你 hook 进 Agent 循环，查看每一步发生了什么。这对于调试和理解 Agent 的工作原理非常有用。

**Agent 循环：**
User Input --> [before_model] --> [wrap_model_call] --> Model --> [after_model] --> Tools --> ...

**我们将构建一个日志记录器，在以下步骤打印信息：**
- **Before model** - 当前对话中有多少条消息？
- **Wrap model call** - 正在使用哪个模型和工具？
- **After model** - 模型是调用了工具还是给出了最终答案？

这就像是添加了调试用的 `print()` 语句，但以一种干净、可重用的方式！

In [20]:
from langchain.agents.middleware import AgentMiddleware, AgentState, ModelRequest, ModelResponse
from typing import Any, Callable

class RequestLoggerMiddleware(AgentMiddleware):
    """用于调试的模型请求日志记录中间件。"""
    
    def before_model(self, state: AgentState, runtime) -> dict[str, Any] | None:
        """在模型执行前记录日志。
        
        Args:
            state: 当前 Agent 的状态 (包含消息历史等)
            runtime: 运行时上下文
        """
        message_count = len(state.get("messages", []))
        print(f"[模型前] 正在处理 {message_count} 条消息")
        return None  # 不修改状态
    
    def wrap_model_call(
        self, 
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse]
    ) -> ModelResponse:
        """包装模型调用，记录请求详情。
        
        Args:
           request: 即将发送给模型的请求对象
           handler: 执行实际模型调用的函数
        """
        print(f"  [模型请求]")
        print(f"   模型: {request.model if hasattr(request, 'model') else '默认'}")
        print(f"   可用工具: {len(request.tools) if request.tools else 0}")
        
        # 调用实际的模型处理程序
        return handler(request)
    
    def after_model(self, state: AgentState, runtime) -> dict[str, Any] | None:
        """在模型执行后记录日志。"""
        last_message = state["messages"][-1]
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            print(f" [模型后] 模型请求了 {len(last_message.tool_calls)} 个工具调用")
        else:
            print(f" [模型后] 模型提供了最终响应")
        return None  # 不修改状态

In [21]:
# 创建带有日志中间件的 Agent
agent_with_logger = create_agent(
    model=model,
    tools=[explain_concept],
    middleware=[RequestLoggerMiddleware()],
)

### 预期结果

当我们运行带日志记录器的 Agent 时，你会实时看到执行流程：

**第一次迭代:**
1. `[模型前]` - 显示我们开始时的消息数量
2. `[模型请求]` - 显示可用模型和工具
3. `[模型后]` - 模型决定调用 `explain_concept` 工具

**第二次迭代 (工具执行后):**
1. `[模型前]` - 现在消息更多了（包含工具结果）
2. `[模型请求]` -再次显示模型信息
3. `[模型后]` - 模型给出最终答案（不再需要工具）

这让你能够深入了解 Agent 的决策过程。

让我们运行它：

In [22]:
# 运行并观察日志
print("\n" + "=" * 50)
print("运行带有日志记录器的 Agent")
print("=" * 50 + "\n")

result = agent_with_logger.invoke({
    "messages": [{"role": "user", "content": "解释递归"}]
})

print("\n" + "=" * 50)
print("最终响应")
print("=" * 50)
print(result["messages"][-1].content)


运行带有日志记录器的 Agent

[模型前] 正在处理 1 条消息
  [模型请求]
   模型: profile={'max_input_tokens': 200000, 'max_output_tokens': 100000, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True} client=<openai.resources.chat.completions.completions.Completions object at 0x7f6ee66a87c0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f6ee53f6920> root_client=<openai.OpenAI object at 0x7f6ee66a8970> root_async_client=<openai.AsyncOpenAI object at 0x7f6f2828cca0> model_name='o3-mini' model_kwargs={} openai_api_key=SecretStr('**********')
   可用工具: 1
 [模型后] 模型请求了 1 个工具调用
[模型前] 正在处理 3 条消息
  [模型请求]
   模型: profile={'max_input_tokens': 200000, 'max_output_tokens': 100000, 'image_inputs': False, 'au

## 第 4 部分: 结合 Middleware 和 Human-in-the-loop

让我们结合 Human-in-the-Loop 和 Middleware，构建一个生产就绪的 Agent：

In [23]:
# 需要批准的敏感工具
@tool
def delete_database(database_name: str) -> str:
    """删除数据库。这很危险！"""
    
    response = interrupt({
        "action": "delete_database",
        "database_name": database_name,
        "warning": "这将永久删除数据库！",
        "message": "你确定要这样做吗？"
    })
    
    if response.get("confirmed"):
        return f"数据库 '{database_name}' 已被删除（模拟）"
    else:
        return "数据库删除已取消"

# 用于跟踪危险操作的中间件
class SafetyMiddleware(AgentMiddleware):
    """添加安全检查和日志记录的中间件。"""
    
    name = "safety_checker"
    
    def after_model(self, state: AgentState) -> dict[str, Any] | None:
        """检查危险的工具调用。"""
        last_message = state["messages"][-1]
        
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            for tool_call in last_message.tool_calls:
                if "delete" in tool_call["name"].lower():
                    print("   [安全] 检测到危险操作！")
                    print(f"   工具: {tool_call['name']}")
                    print(f"   参数: {tool_call['args']}")
        
        return None

# 创建生产级 Agent
production_agent = create_agent(
    model=model,
    tools=[delete_database],
    middleware=[SafetyMiddleware()],
    checkpointer=MemorySaver()
)

  ### 预期结果：多层安全机制实战

  当我们尝试一个危险操作时，你会看到 **两种** 安全机制被激活：

  **第 1 层 - Middleware 检测:**
  - `[安全] 检测到危险操作！` - 中间件发现 delete 操作
  - 记录工具名称和参数以用于审计追踪

  **第 2 层 - 人工批准 (Interrupt):**
  - Agent 执行在 `interrupt()` 处暂停
  - 向审核人员展示警告信息
  - 除非获得明确批准，否则执行不会继续

  **这就是纵深防御：** Middleware 监控所有操作，而 Interrupts 强制对关键行动进行人工批准。

In [24]:
# 测试组合模式
config_4 = {"configurable": {"thread_id": uuid7()}}

print("\n" + "=" * 50)
print("尝试危险操作")
print("=" * 50 + "\n")

# 运行直到 interrupt
result = production_agent.invoke(
    {
        "messages": [HumanMessage(content="删除 production_db 数据库")]
    },
    config=config_4
)

if "__interrupt__" in result:
    interrupt_info = result["__interrupt__"][0]
    print("\n  需要人工批准：")
    print(f"   {interrupt_info.value['warning']}")
    print(f"   Database: {interrupt_info.value['database_name']}")

print("\n(在真实应用中，人类会在继续之前审查此操作)")


尝试危险操作

   [安全] 检测到危险操作！
   工具: delete_database
   参数: {'database_name': 'production_db'}

  需要人工批准：
   这将永久删除数据库！
   Database: production_db

(在真实应用中，人类会在继续之前审查此操作)


## 核心要点

### Human-in-the-Loop (Interrupts)
- 使用 `interrupt()` 暂停执行
- 必须配合 `checkpointer` 实现持久化
- 使用 `Command(resume=value)` 恢复执行
- 非常适合审批工作流和敏感操作

### Middleware
- **Node-style hooks**: `before_model`, `after_model` - 顺序逻辑、验证、日志
- **Wrap-style hooks**: `wrap_model_call`, `wrap_tool_call` - 完全控制、重试、转换
- **Decorators**: `@dynamic_prompt`, `@before_model`, `@wrap_model_call` 用于快速定义中间件
- **Classes**: 继承 `AgentMiddleware` 以构建复杂、可重用的组件

### 何时使用什么？

**使用 Interrupts 当：**
- 动作需要人工批准
- 你想审查/编辑工具调用
- 需要验证用户输入

**使用 Middleware 当：**
- 需要动态修改 Agent 行为
- 想要添加日志/监控
- 需要执行策略（Token 限制、安全检查）
- 想要根据上下文个性化响应

**Node-style vs Wrap-style:**
- Node-style 用于顺序操作（日志、验证）
- Wrap-style 用于控制流（重试、回退、缓存）

## 练习 (可选)

尝试构建一个 Agent：
1. 拥有一个进行购买操作的工具
2. 使用中间件检查购买金额是否超过 `$1000`
3. 如果超过 `$1000`，使用 interrupt 要求批准
4. 如果低于 `$1000`，自动处理

提示：结合 `before_model` 中间件和条件性的 `interrupt()` 逻辑！

In [25]:
# 你的代码！
# 挑战：构建购买审批 Agent

# @tool
# def make_purchase(item: str, amount: float) -> str:
#     ...
#
# class PurchaseLimitMiddleware(AgentMiddleware):
#     ...

## 📚 LangChain & LangGraph 1.0 迁移总结

### 核心变化回顾

#### 1. LangChain 1.0 主要变化

| 特性 | 0.x | 1.0 | 影响 |
|------|-----|-----|------|
| Agent 创建 | `create_react_agent()` | `create_agent()` | 🔴 必须迁移 |
| 中间件 | ❌ 不存在 | ✅ 完整支持 | 🆕 新功能 |
| Content Blocks | 提供商特定 | 统一 API | 🟡 推荐采用 |
| 命名空间 | 混合 | 核心 + Classic | 🟡 逐步迁移 |
| 结构化输出 | 额外 LLM 调用 | 主循环集成 | 🟢 性能提升 |

#### 2. LangGraph 1.0 主要变化

| 特性 | 0.x | 1.0 | 影响 |
|------|-----|-----|------|
| Interrupt | 静态 `interrupt_before` | 动态 `interrupt()` | 🔴 必须迁移 |
| 状态更新 | 分离的返回值和路由 | `Command` 对象 | 🟡 推荐采用 |
| Checkpointer | 不统一 | 统一接口 | 🟡 推荐采用 |
| 核心 API | 实验性 | 稳定（语义化版本） | 🟢 稳定性提升 |

#### 2. 📊 完整对比表

| 功能 | LangChain/LangGraph 0.x | LangChain/LangGraph 1.0 | 迁移复杂度 |
|------|------------------------|------------------------|-----------|
| **Agent 创建** | `langgraph.prebuilt.create_react_agent` | `langchain.agents.create_agent` | 🟢 简单 |
| **中间件** | ❌ 不支持 | ✅ 完整支持 | 🟡 中等 (新功能) |
| **人机交互** | 静态 `interrupt_before` | 动态 `interrupt()` | 🟡 中等 |
| **状态 + 路由** | 分离的函数 | `Command` 对象 | 🟢 简单 |
| **Content Blocks** | 提供商特定 | 统一 `content_blocks` | 🟢 简单 |
| **命名空间** | 所有功能在 `langchain` | 核心在 `langchain`，遗留在 `langchain-classic` | 🟢 简单 |
| **结构化输出** | 需要额外 LLM 调用 | 主循环内生成 | 🟢 简单 |
| **Checkpointer** | 配置方式不统一 | 统一接口 | 🟢 简单 |
| **核心 Graph API** | 实验性 | 稳定 (语义化版本) | 🟢 无需迁移 |

**复杂度说明**：
- 🟢 **简单**：直接替换导入路径或 API 调用
- 🟡 **中等**：需要理解新概念，重构部分代码
- 🔴 **复杂**：需要重新设计架构

---

### 避免的常见陷阱

#### ❌ 陷阱 1：忘记配置 Checkpointer

```python
# 错误：使用 interrupt 但没有 checkpointer
agent = create_agent(model, [tool_with_interrupt])
result = agent.invoke(...)  # ⚠️ 运行时错误！
```

**解决**：所有使用 `interrupt()` 的 Agent 必须配置 `checkpointer`

---

#### ❌ 陷阱 2：混用旧版和新版 API

```python
# 错误：导入新版但使用旧参数名
from langchain.agents import create_agent
agent = create_agent(
    model,
    tools,
    messages_modifier=SystemMessage("...")  # ⚠️ 参数名错误
)
```

**解决**：使用 `system_prompt="..."` 而不是 `messages_modifier`

---

#### ❌ 陷阱 3：Middleware 修改状态但返回 None

```python
# 错误：修改了 state 但没有返回
def before_model(state, runtime):
    state["messages"].append(...)  # ⚠️ 直接修改无效
    return None

# 正确：返回要更新的状态
def before_model(state, runtime):
    return {"messages": state["messages"] + [...]}
```

**解决**：Middleware 必须**返回**状态更新字典

---

## ✅ 迁移检查清单

### 1. 依赖更新
```bash
# 更新到 1.x 版本
pip install --upgrade langchain>=1.1.0 langgraph>=1.0.0

# 如果使用遗留功能
pip install langchain-classic
```

### 2. 代码迁移步骤

#### Step 1: 替换 Agent 创建
- [ ] 将 `from langgraph.prebuilt import create_react_agent` 改为 `from langchain.agents import create_agent`
- [ ] 添加 `system_prompt` 参数
- [ ] 移除 `messages_modifier`（合并到 `system_prompt`）

#### Step 2: 迁移遗留功能
- [ ] 检查是否使用 `langchain.chains`
- [ ] 检查是否使用 `langchain.retrievers`
- [ ] 检查是否使用 `langchain.hub`
- [ ] 如果使用，安装 `langchain-classic` 并更新导入

#### Step 3: 升级 Human-in-the-Loop
- [ ] 将静态 `interrupt_before` 替换为动态 `interrupt()`
- [ ] 更新恢复逻辑使用 `Command(resume=...)`

#### Step 4: (可选) 添加中间件
- [ ] 识别重复的横切逻辑（日志、PII 过滤等）
- [ ] 实现或使用内置中间件
- [ ] 在 `create_agent()` 中添加 `middleware` 参数

#### Step 5: 测试
- [ ] 运行现有测试套件
- [ ] 验证 checkpointer 行为
- [ ] 验证工具调用
- [ ] 验证流式输出
- [ ] 验证 interrupt/resume 流程

### 3. 性能优化建议
- [ ] 使用 `ToolStrategy` 优化结构化输出
- [ ] 考虑使用 `PostgresSaver` 作为生产 checkpointer
- [ ] 启用 LangSmith 进行监控和调试

---



## 练习参考答案

In [26]:
from langchain.agents.middleware import AgentMiddleware
from typing import Any, Callable
from langgraph.types import interrupt, Command
from langchain_core.tools import tool
from langchain.messages import HumanMessage
from langsmith import uuid7

# ==========================================
# 1. 定义购买工具
# ==========================================
@tool
def make_purchase(item: str, amount: float) -> str:
    """
    执行购买操作的工具。
    """
    return f"支付成功：已为 '{item}' 支付 ${amount}。"

# ==========================================
# 2. 定义购买限制中间件 (修正版)
# ==========================================
class PurchaseLimitMiddleware(AgentMiddleware):
    """
    购买风控中间件。
    """
    
    def wrap_tool_call(self, request: Any, handler: Callable[[Any], Any]) -> Any:
        # 修正点：从 request 对象中解析工具调用信息
        # request.tool_call 是一个包含 name, args 等信息的字典
        tool_call = getattr(request, "tool_call", {})
        tool_name = tool_call.get("name")
        tool_args = tool_call.get("args", {})
        
        # 提取参数
        amount = tool_args.get("amount", 0)
        item = tool_args.get("item", "未知商品")

        # 检查逻辑：必须是 make_purchase 工具，且金额超过 1000
        if tool_name == "make_purchase" and amount > 1000:
            print(f"\n[中间件] ⚠️  风控触发：'{item}' 金额 ${amount} 超过自动批准限额 ($1000)。")
            
            # 触发中断
            approval_result = interrupt({
                "type": "purchase_approval",
                "item": item,
                "amount": amount,
                "reason": "金额超限",
                "message": f"申请购买 '{item}'，金额 ${amount}。是否批准？"
            })
            
            # 检查人工审核结果
            if not approval_result.get("approved"):
                print("[中间件] ❌ 购买申请被人工拒绝。")
                return f"交易取消：管理员拒绝了对 '{item}' 的 ${amount} 购买申请。"
            
            print("[中间件] ✅ 购买申请已获批准，继续执行...")
        
        # 执行原始工具调用
        return handler(request)

# ==========================================
# 3. 创建 Agent
# ==========================================
purchase_agent = create_agent(
    model=model,
    tools=[make_purchase],
    middleware=[PurchaseLimitMiddleware()],
    checkpointer=MemorySaver(),
    system_prompt="你是一个采购助手。收到购买请求时，请调用 make_purchase 工具。"
)

# ==========================================
# 4. 验证与测试
# ==========================================

print("\n" + "="*50)
print("测试场景 1: 小额购买 ($300) -> 应自动通过")
print("="*50)

thread_config_1 = {"configurable": {"thread_id": uuid7()}}

response_1 = purchase_agent.invoke(
    {"messages": [HumanMessage(content="我要买一个机械键盘，价格 300 美元")]},
    config=thread_config_1
)
print(f"Agent 回复: {response_1['messages'][-1].content}")


print("\n" + "="*50)
print("测试场景 2: 大额购买 ($2500) -> 应暂停等待批准")
print("="*50)

thread_config_2 = {"configurable": {"thread_id": uuid7()}}

# 第一次运行：触发中断
print(">>> 步骤 1: 发起请求")
response_2 = purchase_agent.invoke(
    {"messages": [HumanMessage(content="公司需要采购一台新的服务器，价格 2500 美元")]},
    config=thread_config_2
)

# 检查是否中断
if "__interrupt__" in response_2:
    interrupt_info = response_2["__interrupt__"][0].value
    print(f"\n[系统] Agent 已暂停，等待人工指令...")
    print(f"详情: {interrupt_info['message']}")
    
    print("\n>>> 步骤 2: 模拟人工批准 (Resume)")
    final_response = purchase_agent.invoke(
        Command(resume={"approved": True}),
        config=thread_config_2
    )
    print(f"Agent 最终回复: {final_response['messages'][-1].content}")
else:
    print("错误：即预期应该暂停，但 Agent 直接执行完成了。")


测试场景 1: 小额购买 ($300) -> 应自动通过
Agent 回复: 已为您成功购买机械键盘！如果需要其他帮助，请告诉我。

测试场景 2: 大额购买 ($2500) -> 应暂停等待批准
>>> 步骤 1: 发起请求

[中间件] ⚠️  风控触发：'新的服务器' 金额 $2500 超过自动批准限额 ($1000)。

[系统] Agent 已暂停，等待人工指令...
详情: 申请购买 '新的服务器'，金额 $2500。是否批准？

>>> 步骤 2: 模拟人工批准 (Resume)

[中间件] ⚠️  风控触发：'新的服务器' 金额 $2500 超过自动批准限额 ($1000)。
[中间件] ✅ 购买申请已获批准，继续执行...
Agent 最终回复: 服务器采购已成功完成，支付金额为 2500 美元。



## 📚 参考资源

### 官方文档
- [LangChain 1.0 发布说明](https://docs.langchain.com/oss/python/releases/langchain-v1)
- [LangGraph 1.0 发布说明](https://docs.langchain.com/oss/python/releases/langgraph-v1)
- [LangChain 1.0 迁移指南](https://docs.langchain.com/oss/python/migrate/langchain-v1)
- [LangGraph 1.0 迁移指南](https://docs.langchain.com/oss/python/migrate/langgraph-v1)

### 深入阅读
- [Middleware 完整指南](https://docs.langchain.com/oss/python/langchain/middleware)
- [Human-in-the-Loop 模式](https://docs.langchain.com/oss/python/langchain/human-in-the-loop)
- [LangChain 1.0 vs LangGraph 1.0 对比](https://www.clickittech.com/ai/langchain-1-0-vs-langgraph-1-0/)

---